# Do some prep work for the movie hack on Thu

- Test movie making on some very high res data to see if its feasible
- Fiddle with the colormaps

In [1]:
import intake
import xarray as xr
import matplotlib.pyplot as plt
import easygems.healpix as egh

In [2]:
catfn='/home/tmerlis/hackathon/xsh24_scream_main_v2.yaml'
combo_cat = intake.open_catalog(catfn)
cat = combo_cat.xsh24_native(zoom=10)
    
dsn_raw =cat.to_dask()
dsn = dsn_raw.pipe(egh.attach_coords)

/home/jbusecke/.conda/envs/easy25/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


In [3]:
dsn = dsn.isel(time=slice(0,900))
dsn

<xarray.Dataset> Size: 997GB
Dimensions:       (time: 900, cell: 12582912)
Coordinates:
  * time          (time) object 7kB 2019-10-20 03:00:00 ... 2020-02-09 12:00:00
    crs           int64 8B 0
  * cell          (cell) int64 101MB 0 1 2 3 ... 12582909 12582910 12582911
    lat           (cell) float64 101MB 0.0373 0.0746 0.0746 ... -0.0746 -0.0373
    lon           (cell) float64 101MB 45.0 45.04 44.96 ... 315.0 315.0 315.0
Data variables: (12/22)
    40dBz_height  (time, cell) float32 45GB dask.array<chunksize=(24, 262144), meta=np.ndarray>
    CAPE_max      (time, cell) float32 45GB dask.array<chunksize=(24, 262144), meta=np.ndarray>
    CIN_max       (time, cell) float32 45GB dask.array<chunksize=(24, 262144), meta=np.ndarray>
    TMP500_300    (time, cell) float32 45GB dask.array<chunksize=(24, 262144), meta=np.ndarray>
    clivi         (time, cell) float32 45GB dask.array<chunksize=(24, 262144), meta=np.ndarray>
    clwvi         (time, cell) float32 45GB dask.array<chunksize=(24, 262144), meta=np.ndarray>
    ...            ...
    w500          (time, cell) float32 45GB dask.array<chunksize=(24, 262144), meta=np.ndarray>
    w850          (time, cell) float32 45GB dask.array<chunksize=(24, 262144), meta=np.ndarray>
    wind100m      (time, cell) float32 45GB dask.array<chunksize=(24, 262144), meta=np.ndarray>
    wind10mmax    (time, cell) float32 45GB dask.array<chunksize=(24, 262144), meta=np.ndarray>
    wmaxdn_min    (time, cell) float32 45GB dask.array<chunksize=(24, 262144), meta=np.ndarray>
    wmaxup_max    (time, cell) float32 45GB dask.array<chunksize=(24, 262144), meta=np.ndarray>

- Black Background?

In [4]:
# This leads to flickering? WTF?

# import matplotlib.pyplot as plt

# # Set default figure background to black
# plt.rcParams['figure.facecolor'] = 'black'

# # Set default axis (subplot) background to black
# plt.rcParams['axes.facecolor'] = 'black'

# # Optional: Set default text and tick colors to white for visibility
# plt.rcParams['text.color'] = 'white'
# plt.rcParams['axes.labelcolor'] = 'white'
# plt.rcParams['xtick.color'] = 'white'
# plt.rcParams['ytick.color'] = 'white'
# plt.rcParams['axes.edgecolor'] = 'white'

In [5]:
import warnings

# Suppress all warnings from Dask Distributed
warnings.filterwarnings("ignore", module="distributed")

In [6]:
from matplotlib.colors import ListedColormap
import numpy as np

# Create an array of RGBA values from white to transparent white
n_colors = 256
colors = np.ones((n_colors, 4))  # all white, RGBA
colors[:, 3] = np.linspace(1.0, 0.0, n_colors)  # alpha channel goes from 1 to 0

# Create and register the colormap
white_to_transparent = ListedColormap(colors, name='white_to_transparent')

# transparent to white
transparent_to_white = ListedColormap(np.flipud(colors), name='transparent_to_white')

# Define the RGB for orange
orange_rgb = [1.0, 0.5, 0.0]

# Create an array of RGBA values from orange to transparent orange
n_colors = 256
colors = np.ones((n_colors, 4))
colors[:, :3] = orange_rgb  # set RGB to orange
colors[:, 3] = np.linspace(1.0, 0.0, n_colors)  # alpha channel from 1 to 0

# Create and register the colormap
orange_to_transparent = ListedColormap(colors, name='orange_to_transparent')

# transparent to orange
transparent_to_orange = ListedColormap(np.flipud(colors), name='transparent_to_orange')

Surface wind magnitude in Blue with CAPE_max in orange for xsh24_native(zoom=10) (I am not quite sure which model this is). 

In [7]:
from distributed import Client, LocalCluster

In [8]:
cluster = LocalCluster(threads_per_worker=1, n_workers=32)
cluster

LocalCluster(21dbc913, 'tcp://127.0.0.1:40607', workers=32, threads=32, memory=500.00 GiB)

In [9]:
client = Client(cluster)

In [10]:
import streamjoy

# streamjoy.config["batch_size"] = 30
# streamjoy.config["threads_per_worker"] = 

In [11]:
from streamjoy import stream, wrap_matplotlib
import xarray as xr
import numpy as np
import cartopy.crs as ccrs

# cartopy.crs.NearsidePerspective(central_longitude=0.0, central_latitude=0.0, satellite_height=35785831, false_easting=0, false_northing=0, globe=None)

@wrap_matplotlib()
def plot_func(da_time, ds_frame_other, central_lon, central_lat, height, **plot_kwargs):
    # projection = ccrs.Robinson(central_longitude=0)
    projection = ccrs.NearsidePerspective(
        central_lon,
        central_lat,
        height
    )
    fig, ax = plt.subplots(
        figsize=(6, 5), 
        subplot_kw={
            "projection": projection, 
            "facecolor":"gray"
        }, 
        # constrained_layout=True,
        # layout='tight',
        dpi=200, 
    )
    ax.set_global()
    # Remove black outline (frame) by hiding spines
    for spine in ax.spines.values():
        spine.set_visible(False)
    
    mag = (ds_frame_other['uas']**2 + ds_frame_other['vas']**2)**0.5
    hpshow = egh.healpix_show(mag, ax=ax, cmap='Blues', vmax=30, vmin=-30, **plot_kwargs)
    egh.healpix_show(ds_frame_other['CAPE_max'], ax=ax, cmap=transparent_to_orange, vmax=7000)
    cbar = fig.colorbar(
        hpshow, 
        ax=ax, 
        orientation='vertical', 
        pad=0.05, shrink=0.8)
    return fig

st = stream(
    dsn, 
    dim='time', 
    var='time', # this natively only works with a dataarray ( I am only passing the time as a dummy here)
    renderer=plot_func, 
    renderer_iterables=[
        [dsn.isel(time=t) for t in range(dsn.time.size)], # I am passing each timestep again as a daataset!!!
        np.linspace(0, 360, dsn.time.size),
        np.linspace(-20, 20, dsn.time.size),
        np.repeat(35785831, dsn.time.size)
    ], 
    
    max_frames=-1,
)
st.write('combined_variables_globe_full.mp4', client = client)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 32
Total threads: 32,Total memory: 500.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40607,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:33639,Total threads: 1
Dashboard: http://127.0.0.1:43151/status,Memory: 15.62 GiB
Nanny: tcp://127.0.0.1:41243,


2025-05-15 01:59:05,853 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 10.84 GiB -- Worker memory limit: 15.62 GiB
[SUCCESS] 01:59AM: Saved stream to /home/jbusecke/viz/combined_variables_globe_full.mp4.
[WARNING] 01:59AM: The output displayed below could be an older, cached version; click the path above to view the latest.


PosixPath('combined_variables_globe_full.mp4')